In [36]:
import numpy as np
import os
from kan import *
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


导入数据并随机采样

In [38]:
data_loc=r"D:\Data\snow_cover_V2"
x_test=np.load(os.path.join(data_loc,"x_test_d.npy"),mmap_mode="r")
y_test=np.load(os.path.join(data_loc,"y_test_d.npy"),mmap_mode="r")
x_train=np.load(os.path.join(data_loc,"x_train_d.npy"),mmap_mode="r")
y_train=np.load(os.path.join(data_loc,"y_train_d.npy"),mmap_mode="r")
# print("x_test_d's shape is",x_test.shape)
# print("y_test_d's shape is",y_test.shape)
# print("x_train_d's shape is",x_train.shape)
# print("y_train_d's shape is",y_train.shape)
# 进行随机采样，采样 0.1% 的数据
sample_size = int(0.001 * len(x_train))
sample_indices = np.random.choice(len(x_train), size=sample_size, replace=False)
x_train_sample = x_train[sample_indices]
y_train_sample = y_train[sample_indices]

sample_size2 = int(0.001 * len(x_test))
sample_indices2 = np.random.choice(len(x_test), size=sample_size2, replace=False)
x_test_sample = x_test[sample_indices2]
y_test_sample = y_test[sample_indices2]
# print(x_train_sample.shape)  # 输出采样后的数据形状
# print(y_train_sample.shape)
# print(x_test_sample.shape)  
# print(y_test_sample.shape)

制作数据集

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x_train, y_train, x_test, y_test, test_size=0.2, random_state=42):
        """
        初始化数据集，使用x_sample和y_sample划分为训练和测试集
        :param x_sample: 特征的numpy数组
        :param y_sample: 标签的numpy数组
        :param test_size: 测试集比例
        :param random_state: 随机种子
        """
        # # 划分训练集和测试集
        # x_train, x_test, y_train, y_test = train_test_split(
        #     x_sample, y_sample, test_size=test_size, random_state=random_state
        # )

        # 将划分好的数据转为Tensor
        # 存储数据集，使用 torch.tensor 从 NumPy 数组或其他格式转换为张量，并将其移动到指定设备（如 GPU）。
        self.data_dict = {
            'train_input': torch.tensor(x_train, dtype=torch.float32).to(device=device),
            'train_label': torch.tensor(y_train, dtype=torch.float32).to(device=device),
            'test_input': torch.tensor(x_test, dtype=torch.float32).to(device=device),
            'test_label': torch.tensor(y_test, dtype=torch.float32).to(device=device)
        }

    def __len__(self):
        # 返回训练数据的长度
        return len(self.data_dict['train_label'])

    def __getitem__(self, idx):
        # 根据索引返回训练数据和标签
        return {
            'train_input': self.data_dict['train_input'][idx],
            'train_label': self.data_dict['train_label'][idx],
            'test_input': self.data_dict['test_input'][idx % len(self.data_dict['test_input'])],  # 循环访问测试数据
            'test_label': self.data_dict['test_label'][idx % len(self.data_dict['test_label'])]   # 循环访问测试标签
        }


In [ ]:
# 实例化数据集
dataset = CustomDataset(x_train_sample,y_train_sample,x_test_sample,y_test_sample)
# 创建数据加载器
dataloader = DataLoader(dataset, batch_size=500, shuffle=True)

# print(dataloader)
# # 验证数据集
# for batch_data in dataloader:
#     print(batch_data["test_label"])
#     break  # 只打印第一个批次

初始化

In [ ]:
# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[6,3,1], grid=5, k=3, seed=0,device=device)
model(dataset.data_dict["train_input"])
model.plot(beta=100)

标准

In [ ]:
# 计算训练集的平均误差（MAE）
def train_mae():
    # 计算预测值和真实值之间的绝对误差
    errors = torch.abs(model(dataset.data_dict['train_input'])[:, 0] - dataset.data_dict['train_label'][:, 0])
    # 计算绝对误差的平均值
    return torch.mean(errors)

# 计算测试集的平均误差（MAE）
def test_mae():
    # 计算预测值和真实值之间的绝对误差
    errors = torch.abs(model(dataset.data_dict['test_input'])[:, 0] - dataset.data_dict['test_label'][:, 0])
    # 计算绝对误差的平均值
    return torch.mean(errors)

#计算均方差（MSE）
def train_mse():
    errors = (model(dataset.data_dict['train_input'])[:, 0] - dataset.data_dict['train_label'][:, 0]) ** 2
    return torch.mean(errors)

def test_mse():
    errors = (model(dataset.data_dict['test_input'])[:, 0] - dataset.data_dict['test_label'][:, 0]) ** 2
    return torch.mean(errors)


第一次训练

这个训练有几个超参数是需要调整的，还有prun减枝的时候那个阈值也需要调整。

lamb、lamb_entropy这个都是正则化参数可以看文档 

[API 6: Training Hyperparamters](https://github.com/KindXiaoming/pykan/blob/master/tutorials/API_demo/API_6_training_hyperparameter.ipynb)

lamb、lamb_entropy都是越大，越稀疏化

然后就是这个默认的learning rate（lr）为1.0很大，看看要不要调小点。



In [ ]:
# train the model
res=model.fit(dataset.data_dict, opt="LBFGS", steps=20, lamb=0.01, lamb_entropy=10, metrics=(train_mae,train_mse,test_mae,test_mse))
train_loss=res["train_loss"]
test_loss=res["test_loss"]

train_mae=res["train_mae"][-1]
train_mse=res["train_mse"][-1]
test_mae=res["test_mae"][-1]
test_mse=res["test_mse"][-1]
# 打印结果
print(f"Train MAE: {train_mae:.4f}")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MSE: {test_mse:.4f}")

# 绘制损失图
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss', color='blue')
plt.plot(test_loss, label='Test Loss', color='orange')
plt.title('Train and Test Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

#绘制模型
model.plot()

In [ ]:
model.prune()
model.plot()

第二次训练+网格细化（第三次训练？？？）+第四次训练

In [ ]:
# train the model
res=model.fit(dataset.data_dict, opt="LBFGS", steps=20, lamb=0.01, lamb_entropy=10, metrics=(train_mae,train_mse,test_mae,test_mse))
train_loss=res["train_loss"]
test_loss=res["test_loss"]

train_mae=res["train_mae"][-1]
train_mse=res["train_mse"][-1]
test_mae=res["test_mae"][-1]
test_mse=res["test_mse"][-1]
# 打印结果
print(f"Train MAE: {train_mae:.4f}")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MSE: {test_mse:.4f}")

# 绘制损失图
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss', color='blue')
plt.plot(test_loss, label='Test Loss', color='orange')
plt.title('Train and Test Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

#绘制模型
model.plot()

In [ ]:
model = model.refine(10)

In [ ]:
# train the model
res=model.fit(dataset.data_dict, opt="LBFGS", steps=20, lamb=0.01, lamb_entropy=10, metrics=(train_mae,train_mse,test_mae,test_mse))
train_loss=res["train_loss"]
test_loss=res["test_loss"]

train_mae=res["train_mae"][-1]
train_mse=res["train_mse"][-1]
test_mae=res["test_mae"][-1]
test_mse=res["test_mse"][-1]
# 打印结果
print(f"Train MAE: {train_mae:.4f}")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MSE: {test_mse:.4f}")

# 绘制损失图
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss', color='blue')
plt.plot(test_loss, label='Test Loss', color='orange')
plt.title('Train and Test Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

#绘制模型
model.plot()

符号回归

In [ ]:
mode = "auto" # "manual"

if mode == "manual":
    # manual mode
    model.fix_symbolic(0,0,0,'sin');
    model.fix_symbolic(0,1,0,'x^2');
    model.fix_symbolic(1,0,0,'exp');
elif mode == "auto":
    # automatic mode
    lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
    model.auto_symbolic(lib=lib)

In [ ]:
from kan.utils import ex_round

SR=ex_round(model.symbolic_formula()[0][0],4)
print(SR)